# Ch. 6. Algorithm Analysis 

## I. Intro to Parallel Algorithm Analysis
This course assumes you have done some computer science, but does not assume any knowledge of algorithm design or formal algorithm analysis. However, I still want to do some basic analysis of some parallel algorithms and serial algorithms, so you have some understanding of what kinds of things will make your code faster, and what kind of code can even be made faster. We're going to be very informal about our algorithm analysis because it's more fun that way, and we will still get a few important points across. After this chapter, I want you to have some intuition that helps you understand why this graph looks the way it does:
![parallel time graph](https://ars.els-cdn.com/content/image/1-s2.0-S0098300413001465-gr9.jpg)
So, let's get started. We talked, as far back as Ch. 3, about what a parallel algorithm looks like and what makes an algorithm parallel. We talked about trivially parallelizable algorithms and monte carlo simulations. What we didn't talk about, specifically, is how these different algorithms change how workflows run. Essentially, if something is trivially parallelizable, if you go from 1 core to _n_ cores, without changing any parameters, you should see a roughly _n_ times speedup. This means that the run time of this program should be roughly $\frac{1}{n}\times (original\ running\ time)$. This means, as you add more cores to your trivially parallelizable code, you should see your code's runtime shrink as $\frac{1}{n}$. This graph is what that looks like:
![1/n](https://i.imgur.com/MbqqIAn.png)

This graph assumes that your code initally took 10 seconds to run, and by the time you add five cores, you already only take ten seconds to run your code.

### Example 6.1 - Monte Carlo Analysis
Given what we just talked about, Monte Carlo simulations should fit nicely into that class of algorithms. We should see a linear speedup and a resulting runtime proportional to $\frac{1}{n}$. In this example, we're going to investigate that and try to show with empirical data that this holds for this case. This should hopefully complement our theoretical analysis from earlier and help convince you that this makes sense. We're going to use our parallel monte carlo pi from earlier, and time it as we scale it.

In [ ]:
# Defining parallel monte carlo pi calculation 

import random
import multiprocessing
from multiprocessing import Pool


#caculate the number of points in the unit circle
#out of n points
def monte_carlo_pi_part(n):
    
    count = 0
    for i in range(n):
        x=random.random()
        y=random.random()
        
        # if it is within the unit circle
        if x*x + y*y <= 1:
            count=count+1
        
    #return
    return count  

In [ ]:
import time
# Note that this cell takes a while to run

n = 100000000

times = []
# Call the process with different sized pools
for np in range(1,32, 2):
    start = time.time()
    part_count=[n//np for i in range(np)]
    pool = Pool(processes=np)   
    # parallel map
    count=pool.map(monte_carlo_pi_part, part_count)
    pi = sum(count)/(n*1.0)*4
    end = time.time()
    tot = end - start
    times.append(tot)
    print("Esitmated value of Pi: {} ({} cores, time: {}s)".format(pi, np, tot))

In [ ]:
%matplotlib inline
# Plot the times we collected
import matplotlib.pyplot as plt
import numpy as np

#h Helper function to plot an equation
def graph(formula, x_range):  
    x = np.array(x_range)  
    y = formula(x) 
    plt.plot(x, y)  

# Graph theoretical maximum (in blue)
graph(lambda x: 32/x, range(1, 32))

# Graph empirical data (in orange)
plt.plot([i*2 for i in range(1,17)], times)

plt.ylabel("Execution Time (sec)")
plt.xlabel("Number of cores")
plt.title("Execution Time vs Number of Cores for Monte Carlo Pi")
plt.show()
# Hopefully, this looks a little like a nice 1/x curve. If not, something is probably broken

## II. Strictly Serial Algorithms
We've seen how algorithms that respond very well to scale do, now what about the other end of the spectrum? If there are algorithms that can be parallelized really easily, it seems like there _should_ be some that can not. Why? Well, it just feels like it! While that's not really valid logic, the assumption is correct. There's a family of algorithms that can simply not be parallelized. This family is called _strictly serial algorithms_, and they are more common than you might think. One such common example is the Fibonacci sequence. The Fibonacci sequence is defined recursively, such that the _n_-th number in the sequence is the sum of the _n-1_-th and the _n-2_-th. That is to say that _fib(n)_ = _fib(n-1)+fib(n-2)_. Because of this, dependence on the previous values, computing the _n_-th Fibonacci number is a strictly serial task. (This is not actually true, as the Fibonacci sequence does indeed have a closed form solution, but that's not the point of this exercise. There are sequences which only have recursive forms, but they're more complicated). This image shows how Fibonacci number computations are always recursively broken down into calls of `fib(1)` or `fib(o)`:
![fib call tree](https://i.stack.imgur.com/7iU1j.png)

### Example 6.2 - Fibonacci Sequence Generator
Because we mentioned how the Fibonacci sequence is a strictly serial algorithm, we're now going to use it as an example. We will write a recursive Fibonacci sequence generator and perform some analysis.

In [ ]:
# Compute Nth fibonacci number
def fib(n):
    if n==0 or n==1:
        return 1
    return fib(n-1) + fib(n-2)

In [ ]:
import time
# Feel free to play around with this
times = []
for i in range(0, 40, 2):
    before = time.time()
    res = fib(i)
    after = time.time()
    tot = after-before
    print("fib({}) = {} ({}sec)".format(i, res, tot))
    times.append(tot)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# Plot the times we collected

plt.plot([2*i for i in range(20)], times)
plt.ylabel("Execution Time (sec)")
plt.xlabel("Number in Fibonacci sequence")
plt.title("Execution Time vs Number in Fibonacci Sequence")
plt.show()

Hopefully, you can see that attempting to parallelize any computation like this is hopeless. 

Let's examine the call tree for calculating the fifth fibonacci number:
```
fib(5) -> fib(4)+fib(3)
            |      |
      fib(3)+fib(2)|
                fib(2)+fib(1)
and so on
```

Now, the execution will still be sequential. This is because, even if you could fork a thread for each call in the tree, it would spawn so many new threads that it simply wouldn't be worth it to do that. It would slow the program down so much that it would end up slower than the serial algorithm you started with.  

In general, you want to parallelize only "embarrassingly parallel" tasks. That is, tasks which are computationally expensive and can be computed independently. Many people forget about the first part. Threads are so expensive that it only makes sense to make one when you have a huge amount of work for them to do, and moreover, that you can devote an entire processor to the thread. If you have 8 processors then making 80 threads is going to force them to share the processor, slowing each one of them down tremendously. You'd do better to make only 8 threads and let each have 100% access to the processor when you have an embarrassingly parallel task to perform.

In this case, it is theoretically possible to parallelize the algorithm (slightly), but practically not worth it. In some cases, such as non primitive recursive problems (like [the Ackermann function](https://en.wikipedia.org/wiki/Ackermann_function)), it's completely impossible. 

Also, note that this could be sped up by a lot through the use of [dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming), but that's not what this course is about.

## III. Task Parallelism vs. Data Parallelism
As you may gather from the title of this section, there are two flavors of parallelism which are important: task parallelism and data parallelism. Do you remember when we talked about Single Instruction, Multiple Data (SIMD)? Well, if you did (or if you didn't), you might be able to guess that SIMD is an example of data parallelism. Let's define data parallelism better. 

Data parallelism focuses on distributing the data across different nodes, which operate on the data in parallel. It can be applied on regular data structures like arrays and matrices by working on each element in parallel. It contrasts to task parallelism as another form of parallelism, where tasks are distributed rather than data (we'll get to this in a moment). 

As this definition suggests, data parallelism occurs when the same thing needs to happen to many different pieces of input data. This is a nice thing to have happen, because it is generally easy to make happen in parallel. Data parallelism is something you should look for when you're trying to speed up your code. It is easy to parallelize, and many different types of problems can be reduced to them.

This image represents a data-parallel process:
![data parallel job](https://upload.wikimedia.org/wikipedia/commons/a/a7/Sequential_vs._Data_Parallel_job_execution.png)
The top describes a job happening in serial, while the bottom shows how one would divide up the top job into the four jobs represented on the bottom, but producing identical output.

Now, let's get into task parallelism. What is task parallelism? Well, task parallelism is a form of parallelism where data is not distributed, but instructions are. Using our acronyms from before, it would be Multiple Instruction, Single Data (MISD). Task parallelism focuses on distributing tasks across different processors. In contrast to data parallelism which involves running the same task on different components of data, task parallelism is distinguished by running many different tasks at the same time on the same data.

In a multiprocessor system, task parallelism is achieved when each processor executes a different thread (or process) on the same or different data. The threads may execute the same or different code. In the general case, different execution threads communicate with one another as they work, but is not a requirement. Communication usually takes place by passing data from one thread to the next as part of a workflow.

As a simple example, if a system is running code on a 2-processor system (CPUs "a" & "b") in a parallel environment and we wish to do tasks "A" and "B", it is possible to tell CPU "a" to do task "A" and CPU "b" to do task "B" simultaneously, thereby reducing the run time of the execution.

The image below represents a comparison between task and data parallelism, with task parallelism on the left and data parallelism is on the right.

![parallelism comparison](http://lh3.googleusercontent.com/8hC1WoryRSmlqL7iQ3VtQwRnCLeO2Wa5zejBw3pmUlXxLpoqhRC2OhNPLZTPfz5OlD4tdmt7V6YqXqBqYmSgg_CQakRHs6IGbwtaliLuUqj7pM6_gCcSbZKSnwoUXaVZnZ8s2pWz4nVxTDnovHjjTxCdpu7p6jly4-Z7ddBH5G43k7ffP8Y-3tXL-o-4UJ-7LJj62Oh7oHEOybzfuw1_MfbDN6N_utlGwuWh1rvy97Niu9Oy9EUS4E7t4N6aJkYUlZptNY7bxC8i0MixcXnCxkH0PtRv7_P-eubpTl2VyKFBEqPyOTsq_8rpiSLsD0vVNjMB7erjSihoNpTI24891t-TOgkJIb2ShZjzUVeyJQQqT2f7tkLCGYRfr8E2V60evnzFw5jyvr06tmImsAvtL2jvo7mduggeusz7HGu6KpUL1APfqt9hFqeJIZOYxUWc-lk7tJ_-XVvhiU4ouwDsKJB2ZhSt_leZODmmE76JtFBSnikFv4ip4YHlt3TwVcsjYF5QNTDHV_rX7CupI2ODOV9gV7MHE6bEkd06edUP7kclqojT1H4C1OUS_yHqHu-3aEpOkuzF-65KGLZ9gRIQL-hyLAGcKJw=w300-h225-no)

### Example 6.3 - Basic Task and Data Parallelism
In this example, we will design one workflow that exhibits task parallelism and one that exhibits data parallelism. A nice real world example of something that is not only a nice simple example of a parallel algorithm, but is also an example of a real world computational task that comes up a lot is matrix multiplication. This image displays how data parallelism comes up in the problem of matrix multiplication:
![matrix multiplication data parallelism](https://upload.wikimedia.org/wikipedia/commons/6/68/Data_Parallelism_in_matrix_multiplication.png)
You can see that after some basic serial computation to set up the right hand side of the equation, each one of the cells in the 3x2 matrix on the right hand side can be computed concurrently or in parallel. This represents data parallelism because the same multiplication and addition instructions are used on different data. For multiplication, we can divide matrix A and B into blocks along rows and columns respectively. This allows us to calculate every element in matrix C individually thereby making the task parallel. 

For example: $A(m \times n) \cdot B (n \times k)$ can be finished in $O(n)$ time instead of $O(m*n*k)$, when executed in parallel using $m*k$ processors. (This is the example from the image).

As our example of data parallelism, we are going to use 

## IV. Maximum Speedups

### Example 6.4 - Timing Fibonacci and Monte Carlo

## V. Real World Middle Ground Examples

### Example 6.5 - 

## Exercise 6. Pi Over Many Nodes or Fibonacci Over Many Nodes
Using Parsl, implement a fibonacci sequence generator and a mote carlo pi simulation. Run it on the entire cluster. Which one runs faster? Which one seems to scale better?

In [ ]:
# Your code goes here